In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24385
2,Huelva,Confirmados PDIA 14 días,814
3,Huelva,Tasa PDIA 14 días,"158,62189917571175"
4,Huelva,Confirmados PDIA 7 días,269
5,Huelva,Total Confirmados,24613
6,Huelva,Curados,14508
7,Huelva,Fallecidos,278


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24385.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6510.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 2055 personas en los últimos 7 días 

Un positivo PCR cada 575 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24385.0,269.0,814.0,513170.0,52.419276,158.621899,59.0
Huelva-Costa,14873.0,158.0,513.0,289548.0,54.567809,177.172697,38.0
Condado-Campiña,6781.0,99.0,252.0,156231.0,63.367706,161.299614,19.0
Huelva (capital),6510.0,70.0,250.0,143837.0,48.666199,173.807852,12.0
Lepe,1978.0,24.0,79.0,27880.0,86.083214,283.357245,11.0
Cartaya,922.0,21.0,62.0,20083.0,104.566051,308.718817,5.0
Moguer,814.0,20.0,39.0,21867.0,91.462020,178.350940,5.0
Bollullos Par del Condado,598.0,16.0,42.0,14387.0,111.211510,291.930215,4.0
Punta Umbría,653.0,11.0,23.0,15355.0,71.637903,149.788343,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Escacena del Campo,138.0,4.0,9.0,2287.0,174.901618,393.528640,0.0
Villanueva de los Castillejos,149.0,5.0,10.0,2825.0,176.991150,353.982301,2.0
Villablanca,171.0,8.0,10.0,2885.0,277.296360,346.620451,2.0
Cartaya,922.0,21.0,62.0,20083.0,104.566051,308.718817,5.0
Bollullos Par del Condado,598.0,16.0,42.0,14387.0,111.211510,291.930215,4.0
Lepe,1978.0,24.0,79.0,27880.0,86.083214,283.357245,11.0
Almonte,835.0,21.0,66.0,24507.0,85.689803,269.310809,0.0
Rociana del Condado,308.0,8.0,21.0,7939.0,100.768359,264.516942,3.0
Moguer,814.0,20.0,39.0,21867.0,91.462020,178.350940,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Aljaraque,916.0,2.0,15.0,21474.0,9.313589,69.851914,1.0,0.133333
Isla Cristina,1975.0,5.0,35.0,21393.0,23.372131,163.604917,2.0,0.142857
Valverde del Camino,414.0,1.0,6.0,12750.0,7.843137,47.058824,0.0,0.166667
Sierra de Huelva-Andévalo Central,2301.0,6.0,28.0,67391.0,8.903266,41.548575,1.0,0.214286
Aracena,453.0,1.0,4.0,8255.0,12.113870,48.455482,0.0,0.250000
San Bartolomé de la Torre,191.0,1.0,4.0,3761.0,26.588673,106.354693,0.0,0.250000
Gibraleón,508.0,4.0,15.0,12737.0,31.404569,117.767135,0.0,0.266667
Huelva (capital),6510.0,70.0,250.0,143837.0,48.666199,173.807852,12.0,0.280000
Palma del Condado (La),962.0,3.0,10.0,10801.0,27.775206,92.584020,1.0,0.300000
